In 1998 Doug Gray wrote [a short paper](https://www.kilohotel.com/rv8/rvlinks/doug_gray/TAS_FNL4.pdf) describing a fantastic method for finding true airpseed vectors (airspeed and heading) from GPS vectors (groundspeed and ground track). Three GPS ground speed/track pairs in any 3 directions is about as easy as can be for a test flight.

Although his spreadsheet formulas work, I wanted to understand it more deeply and write some code. I re-learned some trig and geometry so I could re-derive what is going on here.

## Reference diagram
![Diagram](Diagram.png)

In [ ]:
# preamble
!pip3 install numpy
import numpy as np
import numpy.linalg as la

def cartesian(rho, theta):
    x = rho * np.cos(theta)
    y = rho * np.sin(theta)
    return x, y

def polar(v):
    x, y = v
    rho = np.hypot(x, y)
    theta = np.arctan2(y, x)
    return rho, theta
    
def fromCompass(speed, track):
    theta = np.radians(90 - track)
    return cartesian(speed, theta)

def toCompass(v):
    speed, theta = polar(v)
    track = (90 - np.degrees(theta)) % 360
    return speed, track

def mag(v):
    return np.sqrt(v.dot(v))
    
def angleBetween(u, v):
    return np.arccos(np.dot(u, v) / (mag(u) * mag(v)))

def angle(v):
    x, y = v
    return np.arctan2(y, x)

In [2]:
# flight test data (airspeed, compass heading in degrees)
# Note: these should be in clockwise order
data = [
        (140, 192),
        (112, 283),
        (120, 20)
]

In [3]:
def calculate(data):
        gs1, gs2, gs3 = (fromCompass(speed, track) for speed, track in data)
        # Points a, b, c are the tails of gs1, gs2, gs3 respectively
        ab = np.subtract(gs1, gs2)
        ac = np.subtract(gs1, gs3)
        bc = np.subtract(gs2, gs3)
        abc = angleBetween(ab, bc)
        # AOC = 2*ABC by the inscribed angle theorem.
        # The angle between TAS_C and P is AOC/2=ABC
        # because triangle AOC is isoceles and thus P bisects AOC.
        tas = (mag(ac) / 2) / np.sin(abc)
        p = cartesian(tas * np.cos(abc), angle(ac) + np.radians(90))
        tas1 = np.subtract(np.divide(ac, 2), p)
        w = np.subtract(gs1, tas1)
        tas2 = np.subtract(gs2, w)
        tas3 = np.subtract(gs3, w)

        return tas, w, tas1, tas2, tas3

Doug's results:
- TAS: 130
- Headings: 200°, 287.8°, 11.7°
- Wind: (20.6, 314.8°)


In [4]:
tas, w, tas1, tas2, tas3 = calculate(data)
print("\nOur results:\n")
print(f"  TAS: {tas}")
print(f"  Headings: {[theta for rho, theta in [toCompass(tas1), toCompass(tas2), toCompass(tas3)]]}")
# Wind is stated by where it is coming *from*
print(f"  Wind: {toCompass(-w)}")


Our results:

  TAS: 129.99852349653085
  Headings: [199.67059374146783, 287.792125444477, 11.713025864830456]
  Wind: (20.633444022956596, 314.7584466666683)


## References
- [Using GPS to accurately establish True Airspeed (TAS)](https://www.kilohotel.com/rv8/rvlinks/doug_gray/TAS_FNL4.pdf), Doug Gray, 1998
- [Flight Testing: Finding TAS from GPS Data](https://www.kitplanes.com/flight-testing-finding-tas-from-gps-data/), Kevin Horton, 2009